In [8]:
import os
from langchain_groq import ChatGroq
import getpass
### Import Necessary LangChain Components
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
import requests
from langchain.tools import tool
from bs4 import BeautifulSoup
import requests
import asyncio
import aiohttp
import os
from langchain_groq import ChatGroq
import getpass


if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

# Initialize LangChain's ChatGroq Model
llm = ChatGroq(model="deepseek-r1-distill-llama-70b",temperature=0.5)

# Example Query
response = llm.invoke("Please tell me it is worth buying IBM stock and provide me with some insights on the company.")    
print(response.content)


    

    


<think>
Okay, so I'm trying to figure out whether it's worth buying IBM stock. I don't know much about investing, but I'll try to break it down step by step.

First, I remember that IBM is a big company, often called "Big Blue." They've been around for a long time, so that might mean they're stable. I think they do a lot with computers and technology. But I'm not sure what exactly they're focused on now. Maybe I should look into their current business areas.

I heard something about IBM splitting into two companies a while back. I think one part is IBM and the other is Kyndryl. So, what does IBM focus on now? Maybe they're more into services or cloud computing? I should check that.

I also remember that IBM bought Red Hat a few years ago. Red Hat is known for Linux and open-source software, so maybe IBM is pushing into cloud and enterprise solutions. That could be a good area because cloud computing is growing.

I should consider their financials. How has IBM been doing financially? Ar

In [3]:
pip install langchain-deepseek

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langchain-deepseek (from versions: none)
ERROR: No matching distribution found for langchain-deepseek
You should consider upgrading via the 'f:\Project\LLMStockPulse-main\myvenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
from langchain.chat_models import ChatDeepSeek  # Import DeepSeek mode

ImportError: cannot import name 'ChatDeepSeek' from 'langchain.chat_models' (f:\Project\LLMStockPulse-main\myvenv\lib\site-packages\langchain\chat_models\__init__.py)

In [6]:
import os
import getpass
from langchain.llms import BaseLLM
import requests

class ChatDeepSeek(BaseLLM):
    def __init__(self, temperature=0.7):
        self.temperature = temperature
        self.api_key = os.environ.get("DEEPSEEK_API_KEY")
        self.endpoint = "https://api.deepseek.ai/v1/chat"  # Replace with actual DeepSeek endpoint
    
    def _call(self, prompt: str) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }
        payload = {
            "model": "chatgpt",  # Specify the model you want
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature,
        }
        response = requests.post(self.endpoint, json=payload, headers=headers)
        
        if response.status_code == 200:
            return response.json().get("choices", [{}])[0].get("message", {}).get("content", "No response")
        else:
            return f"Error: {response.status_code} - {response.text}"

# Example usage
llm = ChatDeepSeek(temperature=0.5)

response = llm("What is LangChain?")
print(response)


TypeError: Can't instantiate abstract class ChatDeepSeek with abstract methods _generate, _llm_type

In [ ]:
import os
import getpass
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import tool
import requests
from bs4 import BeautifulSoup
import asyncio
import aiohttp
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss  # For fast similarity search

# Ensure the API Key is set
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

# Initialize LangChain's ChatGroq Model
llm = ChatGroq(temperature=0)


if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")
    
def process_search_tool(url: str) -> str:
    """Fetches and extracts content from a given URL."""
    try:
        # Add headers to mimic a real browser
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.9",
            "Referer": "https://www.google.com/"
        }
        response = requests.get(url, headers=headers, timeout=5)
        response.raise_for_status()  # Raise an error for bad status codes
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract text from specific tags to avoid noise
        text = " ".join([p.get_text() for p in soup.find_all(["p", "h1", "h2", "h3", "article"])])
        return text
    except requests.Timeout:
        return f"Timeout error while processing URL {url}"
    except Exception as e:
        return f"Error processing URL {url}: {str(e)}"

# Asynchronous function to process multiple URLs concurrently
async def process_multiple_urls(urls):
    loop = asyncio.get_event_loop()
    
    # Use partial to pass the function and its arguments
    tasks = [loop.run_in_executor(None, partial(process_search_tool, url)) for url in urls]
    results = await asyncio.gather(*tasks)
    return results



# Initialize Sentence-BERT Model for Embedding Generation
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example Query to Encode
query = "Tell me top ten trending current news in the world"

# Generate Embedding for Query
query_embedding = sbert_model.encode([query])[0]
print(query_embedding)

def tavily_search(query: str) -> list:
    """Fetches search results for a given query using Tavily."""
    tavily_search = TavilySearchResults(max_results=5, search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True)
    return tavily_search.run(query)

# Example tool use for searching data
search_results = tavily_search(query)

# Use the retrieved search results to encode their embeddings
search_embeddings = sbert_model.encode(search_results)
print(search_embeddings)

# Now, let's set up FAISS for fast similarity search
index = faiss.IndexFlatL2(query_embedding.shape[0])  # Using L2 distance (Euclidean distance)
print(index)
index.add(np.array(search_embeddings))  # Add search embeddings to the FAISS index
print(index)

# Perform the similarity search (retrieve top-k most similar search results)
k = 3  # Number of similar results you want to fetch
D, I = index.search(np.array([query_embedding]), k)  # D: distances, I: indices of nearest neighbors
print(D, I)

# Fetch the most relevant results based on the indices
relevant_results = [search_results[i] for i in I[0]]
print(relevant_results)

# Now you can integrate these results as context to refine the query
query_with_context = f"Based on this information: {relevant_results}"

# Run the refined query with additional context through ChatGroq
response_with_context = llm.invoke(query_with_context)
print(response_with_context.content)


[-3.48276466e-01 -3.67820472e-01  1.85436174e-01 -5.02136052e-01
  6.02458954e-01  3.72113526e-01 -4.41867650e-01  3.83968204e-01
 -8.18194225e-02  3.17060039e-04 -3.18080306e-01  6.51477873e-02
 -4.97912429e-02  2.53298640e-01 -5.96161261e-02 -2.42521361e-01
  6.79208159e-01 -5.30129373e-01  5.14001660e-02 -2.72889733e-01
 -5.24730086e-01 -1.76243648e-01  3.66427302e-01 -1.57511339e-01
  1.82454899e-01 -3.53101283e-01 -1.92305416e-01  7.65354559e-02
 -1.01229839e-01 -1.11638813e-03 -2.72053570e-01  2.08752602e-01
  1.17489502e-01  2.86106944e-01 -3.79813403e-01 -2.78891921e-01
  4.18795526e-01  1.03510603e-01  4.77114357e-02  2.60568380e-01
 -3.33941262e-03 -6.08178079e-01 -1.44443791e-02 -1.44871220e-01
 -2.54031718e-01  1.07129879e-01 -2.60808039e-03  6.04766130e-01
 -2.23087575e-02  2.94883281e-01 -3.81902516e-01 -2.56321907e-01
  2.01145425e-01  1.29524246e-01  5.54436684e-01 -3.93291980e-01
 -4.43785816e-01  6.78732097e-02  6.94405586e-02 -3.28105122e-01
  1.56036913e-01 -5.44280

In [30]:
import os
import getpass
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss  # For fast similarity search
import tiktoken

# Ensure the API Key is set
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

# Initialize LangChain's ChatGroq Model
llm = ChatGroq(temperature=0)

# Ensure Tavily API Key is set
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

# Initialize Sentence-BERT Model for Embedding Generation
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Initialize Tavily Search Tool
tavily_tool = TavilySearchResults(max_results=5, search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True)

# Example Query
query = "Tell me what happened between Donald Trump and Zelensky at the Oval office meeting?"

# Step 1: Fetch search results using Tavily
search_results = tavily_tool.run(query)

# Step 2: Extract content from search results
search_contents = [result.get("content", "") for result in search_results]

# Step 2: Generate embeddings for the query and search results
query_embedding = sbert_model.encode([query])[0]  # Embedding for the query
search_embeddings = sbert_model.encode([result["content"] for result in search_results])  # Embeddings for search results

# Step 3: Set up FAISS for similarity search
index = faiss.IndexFlatL2(query_embedding.shape[0])  # Using L2 distance (Euclidean distance)
index.add(np.array(search_embeddings))  # Add search embeddings to the FAISS index

# Step 4: Perform similarity search (retrieve top-k most similar search results)
k = 3  # Number of similar results to fetch
D, I = index.search(np.array([query_embedding]), k)  # D: distances, I: indices of nearest neighbors

# Step 5: Fetch the most relevant results based on the indices
relevant_results = [search_results[i] for i in I[0]]

# Step 6: Refine the query with the relevant context
query_with_context = f"Based on this information: {relevant_results}"

# Step 7: Check token count before sending the request
encoding = tiktoken.get_encoding("cl100k_base")
total_tokens = len(encoding.encode(query_with_context))

if total_tokens > 5000:
    print("Request too large. Reducing input size.")
    relevant_results = [result[:100] for result in relevant_results]  # Further trim results
    query_with_context = f"Based on this information: {relevant_results}"

# Step 8: Run the refined query with additional context through ChatGroq
response_with_context = llm.invoke(query_with_context)
print(response_with_context.content)

It appears that there was a heated argument between President Donald Trump, Ukrainian President Volodymyr Zelensky, and Vice President JD Vance during a meeting in the White House Oval Office on February 28, 2025. The argument was significant enough to cause a pause in intelligence sharing between the US and Ukraine, according to Trump officials.

President Zelensky has since described the meeting as "regrettable" and noted that Ukraine is ready to negotiate about an end to the conflict. The exact reason for the argument is not specified in the articles, but it is mentioned that it was a "blowup" and that the meeting "did not go the way it was supposed to."

There are also several images attached to one of the articles showing President Trump and President Zelensky during the meeting.


In [25]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'f:\Project\LLMStockPulse-main\myvenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [2]:
### Import Necessary LangChain Components
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
import requests
from langchain.tools import tool
from bs4 import BeautifulSoup
import requests
import asyncio
import aiohttp
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
# Add this import at the top with other imports
from langchain_core.documents import Document
from langchain.memory import ConversationSummaryMemory
from langchain.chains import LLMChain


API_KEY ="YL41PNDL63AAWZOI"

@tool("stock_api_tool", return_direct=False)
def get_stock_data(stock_symbol: str, data_type: str = "intraday") -> str:
    """
    Fetches stock data for a given stock symbol.
    
    Available data_type options:
    - "intraday": Latest stock price (1-minute interval)
    - "daily": Daily adjusted closing prices
    - "fundamental": Company overview (market cap, EPS, PE ratio)
    - "indicators": Technical indicators (RSI, MACD)
    """
    
    API_KEY = "QYBCUX9XUW8ESTIU"
    BASE_URL = "https://www.alphavantage.co/query"
    params = {"symbol": stock_symbol, "apikey": API_KEY}

    if data_type == "intraday":
        params["function"] = "TIME_SERIES_INTRADAY"
        params["interval"] = "1min"
    elif data_type == "daily":
        params["function"] = "TIME_SERIES_DAILY_ADJUSTED"
    elif data_type == "fundamental":
        params["function"] = "OVERVIEW"
    elif data_type == "indicators":
        params["function"] = "RSI"
        params["interval"] = "daily"
        params["time_period"] = "14"
        params["series_type"] = "close"
    elif data_type == "financials":
        params["function"] = "INCOME_STATEMENT"
    else:
        return "Invalid data type. Choose from 'intraday', 'daily', 'fundamental', 'financials','market_sentiment_news', or 'indicators'."

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    try:
        if data_type == "intraday":
            latest_data = data['Time Series (1min)']
            latest_timestamp = next(iter(latest_data))
            stock_info = latest_data[latest_timestamp]
            return f"Stock: {stock_symbol} - Open: {stock_info['1. open']}, High: {stock_info['2. high']}, Low: {stock_info['3. low']}, Close: {stock_info['4. close']}, Volume: {stock_info['5. volume']} at {latest_timestamp}"
        
        elif data_type == "daily":
            latest_data = data['Time Series (Daily)']
            latest_date = next(iter(latest_data))
            stock_info = latest_data[latest_date]
            return f"Stock: {stock_symbol} - Open: {stock_info['1. open']}, High: {stock_info['2. high']}, Low: {stock_info['3. low']}, Close: {stock_info['4. close']}, Adjusted Close: {stock_info['5. adjusted close']}, Volume: {stock_info['6. volume']} on {latest_date}"
        
        elif data_type == "fundamental":
            return f"Company: {data['Name']} ({stock_symbol})\nMarket Cap: {data['MarketCapitalization']}\nEPS: {data['EPS']}\nPE Ratio: {data['PERatio']}\nDividend Yield: {data['DividendYield']}\nSector: {data['Sector']}"
        
        elif data_type == "indicators":
            rsi_data = data['Technical Analysis: RSI']
            latest_date = next(iter(rsi_data))
            return f"Stock: {stock_symbol} - RSI: {rsi_data[latest_date]['RSI']} on {latest_date}"
        elif data_type == "financials":
            annual_reports = data["annualReports"][0]  # Latest financial year report
            return f"Company: {stock_symbol}\nRevenue: {annual_reports['totalRevenue']}\nNet Income: {annual_reports['netIncome']}\nProfit Margin: {annual_reports['grossProfit']}"


    except KeyError:
        return "Error fetching stock data. Check API limits or verify the stock symbol."
    
@tool("stock_news_api_tool", return_direct=False)
def get_market_sentiment_news(ticker: str = None, topics: str = None):
    """
    Fetches the latest market sentiment news for a given stock ticker or market topics.

    Parameters:
    - ticker (str, optional): Stock symbol (e.g., "AAPL") to fetch news for.
    - topics (str, optional): Market topics such as "Earnings", "IPO", etc.

    Returns:
    - str: Formatted string containing the top 3 news articles with title, source, sentiment, and URL.
    """

    API_KEY = "QYBCUX9XUW8ESTIU"
    BASE_URL = "https://www.alphavantage.co/query"

    params = {
        "function": "NEWS_SENTIMENT",
        "apikey": API_KEY
    }

    if ticker:
        params["tickers"] = ticker  # Fetch news specific to the stock
    if topics:
        params["topics"] = topics  # Filter by topics like Earnings, IPO, etc.

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    news_items = data.get("feed", [])
    if not news_items:
        return "No market news available."

    # Extract top 3 news articles
    news_summary = []
    for news in news_items[:3]:
        news_summary.append(
            f"**Title**: {news['title']}\n"
            f"**Source**: {news['source']}\n"
            f"**Sentiment**: {news['overall_sentiment_label']}\n"
            f"**URL**: {news['url']}\n"
        )

    return "\n".join(news_summary)

@tool("process_search_tool", return_direct=False)
def process_search_tool(url: str) -> str:
    """Fetches and extracts content from a given URL."""
    try:
        # Add headers to mimic a real browser
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.9",
            "Referer": "https://www.google.com/"
        }
        response = requests.get(url, headers=headers, timeout=5)
        response.raise_for_status()  # Raise an error for bad status codes
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract text from specific tags to avoid noise
        text = " ".join([p.get_text() for p in soup.find_all(["p", "h1", "h2", "h3", "article"])])
        return text
    except requests.Timeout:
        return f"Timeout error while processing URL {url}"
    except Exception as e:
        return f"Error processing URL {url}: {str(e)}"

# Asynchronous function to process multiple URLs concurrently
async def process_multiple_urls(urls):
    loop = asyncio.get_event_loop()
    
    # Use partial to pass the function and its arguments
    tasks = [loop.run_in_executor(None, partial(process_search_tool, url)) for url in urls]
    results = await asyncio.gather(*tasks)
    return results



# Function to fetch and return up to 5 search results from Tavily
@tool("tavily_search_tool", return_direct=False)
def tavily_rag_search(query: str) -> list:
    """Enhanced search with RAG processing"""
    tavily = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True
    )
    results = tavily.run(query)
# Convert to LangChain Documents
    documents = [
        Document(
            page_content=res["content"],
            metadata={"source": res["url"], "title": res.get("title", "")}
        ) for res in results
    ]
    
    # Rest of your RAG processing
    search_contents = [doc.page_content for doc in documents]
    query_embedding = sbert_model.encode([query])[0]
    search_embeddings = sbert_model.encode(search_contents)
    
    index = faiss.IndexFlatL2(query_embedding.shape[0])
    index.add(np.array(search_embeddings))
    
    k = 3
    D, I = index.search(np.array([query_embedding]), k)
    
    relevant_docs = [documents[i] for i in I[0]]  # Return Document objects
    
    return relevant_docs


# NewsAPI tool to fetch news articles
@tool("news_api_tool", return_direct=False)
def get_news_from_newsapi(query: str) -> str:
    """
    Fetches the latest news articles from NewsAPI for a specific query.
    """
    API_KEY = "d2afe10169b44e628b2131aed04ac7e4"  # Add your NewsAPI key here
    BASE_URL = "https://newsapi.org/v2/everything"

    params = {
        "q": query,  # Use the query from Tavily search
        "apiKey": API_KEY,
        "language": "en",  # You can adjust the language as needed
        "sortBy": "relevance",  # Sort by relevance or any other criteria
    }

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    # Extract top 3 news articles
    if data.get("status") == "ok":
        articles = data.get("articles", [])
        if articles:
            news_summary = []
            for article in articles[:3]:  # Limit to top 3 articles
                news_summary.append(
                    f"**Title**: {article['title']}\n"
                    f"**Source**: {article['source']['name']}\n"
                    f"**Description**: {article['description']}\n"
                    f"**URL**: {article['url']}\n"
                )
            return "\n".join(news_summary)
    return "No news articles found."


@tool("get_stock_analysis_tool", return_direct=False)
def get_stock_analysis(query: str) -> str:
    """
    Fetches stock financial data, technical indicators, and news sentiment analysis 
    for a given stock symbol and provides a Buy/Hold/Sell recommendation.
    """
    
    try:
        # Fetch stock overview (P/E ratio, market cap, dividend yield)
        overview_url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={API_KEY}"
        stock_data = requests.get(overview_url).json()

        if "Error Message" in stock_data or "Note" in stock_data:
            return f"❌ Error: Could not fetch data for {symbol}. API limit may be reached."

        pe_ratio = float(stock_data.get("PERatio", 0))
        market_cap = float(stock_data.get("MarketCapitalization", 0))
        dividend_yield = float(stock_data.get("DividendYield", 0))

        # Fetch RSI (Relative Strength Index)
        rsi_url = f"https://www.alphavantage.co/query?function=RSI&symbol={symbol}&interval=daily&time_period=14&series_type=close&apikey={API_KEY}"
        rsi_data = requests.get(rsi_url).json()
        rsi_values = rsi_data.get("Technical Analysis: RSI", {})

        # Get latest available RSI value
        latest_rsi_date = next(iter(rsi_values), None)
        rsi_value = float(rsi_values[latest_rsi_date]["RSI"]) if latest_rsi_date else 50  # Default 50 if unavailable

        # Fetch latest stock price
        price_url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={API_KEY}"
        price_data = requests.get(price_url).json()
        stock_price = float(price_data.get("Global Quote", {}).get("05. price", 0))

        # Fetch News Sentiment
        news_url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={API_KEY}"
        news_response = requests.get(news_url).json()
        news_sentiment = news_response.get("feed", [])

        # Compute average sentiment score from top 5 news articles
        total_sentiment = sum(news.get("overall_sentiment_score", 0) for news in news_sentiment[:5])
        avg_sentiment = total_sentiment / max(len(news_sentiment[:5]), 1)  # Avoid division by zero

        # **Decision Logic**
        if rsi_value < 30 and pe_ratio < 20 and avg_sentiment > 0:
            recommendation = "🔵 BUY: The stock is undervalued and news sentiment is positive."
        elif rsi_value > 70 and avg_sentiment < 0:
            recommendation = "🔴 SELL: The stock is overbought and news sentiment is negative."
        else:
            recommendation = "🟡 HOLD: Market conditions are stable."

        return f"""
        📈 **Stock Analysis for {symbol}**
        - **Current Price:** ${stock_price:.2f}
        - **P/E Ratio:** {pe_ratio:.2f}
        - **Market Cap:** ${market_cap:,.0f}
        - **RSI (14-day):** {rsi_value:.2f}  
        - **Dividend Yield:** {dividend_yield:.2%}
        - **News Sentiment Score:** {avg_sentiment:.2f}

        **Recommendation: {recommendation}**
        """

    except Exception as e:
        return f"❌ Error fetching data: {str(e)}"

tools = [get_news_from_newsapi,get_market_sentiment_news,get_stock_data,get_stock_analysis,tavily_rag_search, process_search_tool]
### Create a Chat Prompt Template
# Step 1: Define Summarized Memory
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

# Step 2: Update the Prompt to Use Summarized Memory
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),  # Summarized memory placeholder
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Step 3: Construct the Agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Step 4: Create an Agent Executor with Summarized Memory
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

# Example Queries with Summarization Memory
query1 = "Tell me the stock market performance in NSE?"
response1 = agent_executor.invoke({"input": query1})
print(response1["output"])

query2 = "Give me an update on the same market today."
response2 = agent_executor.invoke({"input": query2})
print(response2["output"])  # Now considers summarized past conversations

query3 = "What were the key trends from my past queries?"
response3 = agent_executor.invoke({"input": query3})
print(response3["output"])  # Provides a summary of past discussions

NameError: name 'llm' is not defined

In [ ]:
### Import Necessary LangChain Components
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
import requests
from langchain.tools import tool
from bs4 import BeautifulSoup
import requests
import asyncio
import aiohttp
import os
from langchain_groq import ChatGroq
import getpass


API_KEY ="YL41PNDL63AAWZOI"

@tool("stock_api_tool", return_direct=False)
def get_stock_data(stock_symbol: str, data_type: str = "intraday") -> str:
    """
    Fetches stock data for a given stock symbol.
    
    Available data_type options:
    - "intraday": Latest stock price (1-minute interval)
    - "daily": Daily adjusted closing prices
    - "fundamental": Company overview (market cap, EPS, PE ratio)
    - "indicators": Technical indicators (RSI, MACD)
    """
   
    BASE_URL = "https://www.alphavantage.co/query"
    params = {"symbol": stock_symbol, "apikey": API_KEY}

    if data_type == "intraday":
        params["function"] = "TIME_SERIES_INTRADAY"
        params["interval"] = "1min"
    elif data_type == "daily":
        params["function"] = "TIME_SERIES_DAILY_ADJUSTED"
    elif data_type == "fundamental":
        params["function"] = "OVERVIEW"
    elif data_type == "indicators":
        params["function"] = "RSI"
        params["interval"] = "daily"
        params["time_period"] = "14"
        params["series_type"] = "close"
    elif data_type == "financials":
        params["function"] = "INCOME_STATEMENT"
    else:
        return "Invalid data type. Choose from 'intraday', 'daily', 'fundamental', 'financials','market_sentiment_news', or 'indicators'."

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    try:
        if data_type == "intraday":
            latest_data = data['Time Series (1min)']
            latest_timestamp = next(iter(latest_data))
            stock_info = latest_data[latest_timestamp]
            return f"Stock: {stock_symbol} - Open: {stock_info['1. open']}, High: {stock_info['2. high']}, Low: {stock_info['3. low']}, Close: {stock_info['4. close']}, Volume: {stock_info['5. volume']} at {latest_timestamp}"
        
        elif data_type == "daily":
            latest_data = data['Time Series (Daily)']
            latest_date = next(iter(latest_data))
            stock_info = latest_data[latest_date]
            return f"Stock: {stock_symbol} - Open: {stock_info['1. open']}, High: {stock_info['2. high']}, Low: {stock_info['3. low']}, Close: {stock_info['4. close']}, Adjusted Close: {stock_info['5. adjusted close']}, Volume: {stock_info['6. volume']} on {latest_date}"
        
        elif data_type == "fundamental":
            return f"Company: {data['Name']} ({stock_symbol})\nMarket Cap: {data['MarketCapitalization']}\nEPS: {data['EPS']}\nPE Ratio: {data['PERatio']}\nDividend Yield: {data['DividendYield']}\nSector: {data['Sector']}"
        
        elif data_type == "indicators":
            rsi_data = data['Technical Analysis: RSI']
            latest_date = next(iter(rsi_data))
            return f"Stock: {stock_symbol} - RSI: {rsi_data[latest_date]['RSI']} on {latest_date}"
        elif data_type == "financials":
            annual_reports = data["annualReports"][0]  # Latest financial year report
            return f"Company: {stock_symbol}\nRevenue: {annual_reports['totalRevenue']}\nNet Income: {annual_reports['netIncome']}\nProfit Margin: {annual_reports['grossProfit']}"


    except KeyError:
        return "Error fetching stock data. Check API limits or verify the stock symbol."
    
@tool("stock_news_api_tool", return_direct=False)
def get_market_sentiment_news(ticker: str = None, topics: str = None):
    """
    Fetches the latest market sentiment news for a given stock ticker or market topics.

    Parameters:
    - ticker (str, optional): Stock symbol (e.g., "AAPL") to fetch news for.
    - topics (str, optional): Market topics such as "Earnings", "IPO", etc.

    Returns:
    - str: Formatted string containing the top 3 news articles with title, source, sentiment, and URL.
    """


    BASE_URL = "https://www.alphavantage.co/query"

    params = {
        "function": "NEWS_SENTIMENT",
        "apikey": API_KEY
    }

    if ticker:
        params["tickers"] = ticker  # Fetch news specific to the stock
    if topics:
        params["topics"] = topics  # Filter by topics like Earnings, IPO, etc.

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    news_items = data.get("feed", [])
    if not news_items:
        return "No market news available."

    # Extract top 3 news articles
    news_summary = []
    for news in news_items[:3]:
        news_summary.append(
            f"**Title**: {news['title']}\n"
            f"**Source**: {news['source']}\n"
            f"**Sentiment**: {news['overall_sentiment_label']}\n"
            f"**URL**: {news['url']}\n"
        )

    return "\n".join(news_summary)

@tool("process_search_tool", return_direct=False)
def process_search_tool(url: str) -> str:
    """Fetches and extracts content from a given URL."""
    try:
        # Add headers to mimic a real browser
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.9",
            "Referer": "https://www.google.com/"
        }
        response = requests.get(url, headers=headers, timeout=5)
        response.raise_for_status()  # Raise an error for bad status codes
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract text from specific tags to avoid noise
        text = " ".join([p.get_text() for p in soup.find_all(["p", "h1", "h2", "h3", "article"])])
        return text
    except requests.Timeout:
        return f"Timeout error while processing URL {url}"
    except Exception as e:
        return f"Error processing URL {url}: {str(e)}"

# Asynchronous function to process multiple URLs concurrently
async def process_multiple_urls(urls):
    loop = asyncio.get_event_loop()
    
    # Use partial to pass the function and its arguments
    tasks = [loop.run_in_executor(None, partial(process_search_tool, url)) for url in urls]
    results = await asyncio.gather(*tasks)
    return results


# Function to fetch and return up to 5 search results from Tavily
@tool("tavily_search_tool", return_direct=False)
def tavily_rag_search(query: str) -> list:
    """Enhanced search with RAG processing"""
    tavily = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True
    )
    results = tavily.run(query)
# Convert to LangChain Documents
    documents = [
        Document(
            page_content=res["content"],
            metadata={"source": res["url"], "title": res.get("title", "")}
        ) for res in results
    ]
    
    # Rest of your RAG processing
    search_contents = [doc.page_content for doc in documents]
    query_embedding = sbert_model.encode([query])[0]
    search_embeddings = sbert_model.encode(search_contents)
    
    index = faiss.IndexFlatL2(query_embedding.shape[0])
    index.add(np.array(search_embeddings))
    
    k = 3
    D, I = index.search(np.array([query_embedding]), k)
    
    relevant_docs = [documents[i] for i in I[0]]  # Return Document objects
    
    return relevant_docs

# NewsAPI tool to fetch news articles
@tool("news_api_tool", return_direct=False)
def get_news_from_newsapi(query: str) -> str:
    """
    Fetches the latest news articles from NewsAPI for a specific query.
    """
    API_KEY = "d2afe10169b44e628b2131aed04ac7e4"  # Add your NewsAPI key here
    BASE_URL = "https://newsapi.org/v2/everything"

    params = {
        "q": query,  # Use the query from Tavily search
        "apiKey": API_KEY,
        "language": "en",  # You can adjust the language as needed
        "sortBy": "relevance",  # Sort by relevance or any other criteria
    }

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    # Extract top 3 news articles
    if data.get("status") == "ok":
        articles = data.get("articles", [])
        if articles:
            news_summary = []
            for article in articles[:3]:  # Limit to top 3 articles
                news_summary.append(
                    f"**Title**: {article['title']}\n"
                    f"**Source**: {article['source']['name']}\n"
                    f"**Description**: {article['description']}\n"
                    f"**URL**: {article['url']}\n"
                )
            return "\n".join(news_summary)
    return "No news articles found."


@tool("get_stock_analysis_tool", return_direct=False)
def get_stock_analysis(query: str) -> str:
    """
    Fetches stock financial data, technical indicators, and news sentiment analysis 
    for a given stock symbol and provides a Buy/Hold/Sell recommendation.
    """
    
    try:
        # Fetch stock overview (P/E ratio, market cap, dividend yield)
        overview_url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={API_KEY}"
        stock_data = requests.get(overview_url).json()

        if "Error Message" in stock_data or "Note" in stock_data:
            return f"❌ Error: Could not fetch data for {symbol}. API limit may be reached."

        pe_ratio = float(stock_data.get("PERatio", 0))
        market_cap = float(stock_data.get("MarketCapitalization", 0))
        dividend_yield = float(stock_data.get("DividendYield", 0))

        # Fetch RSI (Relative Strength Index)
        rsi_url = f"https://www.alphavantage.co/query?function=RSI&symbol={symbol}&interval=daily&time_period=14&series_type=close&apikey={API_KEY}"
        rsi_data = requests.get(rsi_url).json()
        rsi_values = rsi_data.get("Technical Analysis: RSI", {})

        # Get latest available RSI value
        latest_rsi_date = next(iter(rsi_values), None)
        rsi_value = float(rsi_values[latest_rsi_date]["RSI"]) if latest_rsi_date else 50  # Default 50 if unavailable

        # Fetch latest stock price
        price_url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={API_KEY}"
        price_data = requests.get(price_url).json()
        stock_price = float(price_data.get("Global Quote", {}).get("05. price", 0))

        # Fetch News Sentiment
        news_url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={API_KEY}"
        news_response = requests.get(news_url).json()
        news_sentiment = news_response.get("feed", [])

        # Compute average sentiment score from top 5 news articles
        total_sentiment = sum(news.get("overall_sentiment_score", 0) for news in news_sentiment[:5])
        avg_sentiment = total_sentiment / max(len(news_sentiment[:5]), 1)  # Avoid division by zero

        # **Decision Logic**
        if rsi_value < 30 and pe_ratio < 20 and avg_sentiment > 0:
            recommendation = "🔵 BUY: The stock is undervalued and news sentiment is positive."
        elif rsi_value > 70 and avg_sentiment < 0:
            recommendation = "🔴 SELL: The stock is overbought and news sentiment is negative."
        else:
            recommendation = "🟡 HOLD: Market conditions are stable."

        return f"""
        📈 **Stock Analysis for {symbol}**
        - **Current Price:** ${stock_price:.2f}
        - **P/E Ratio:** {pe_ratio:.2f}
        - **Market Cap:** ${market_cap:,.0f}
        - **RSI (14-day):** {rsi_value:.2f}  
        - **Dividend Yield:** {dividend_yield:.2%}
        - **News Sentiment Score:** {avg_sentiment:.2f}

        **Recommendation: {recommendation}**
        """

    except Exception as e:
        return f"❌ Error fetching data: {str(e)}"

# Define specialized agents
def create_stock_data_agent(llm):
    tools = [get_stock_data, get_stock_analysis]
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a stock data expert. Fetch and analyze stock data."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),  # Add agent_scratchpad placeholder
    ])
    return create_tool_calling_agent(llm, tools, prompt)

def create_sentiment_agent(llm):
    tools = [get_market_sentiment_news]
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a sentiment analysis expert. Analyze news sentiment."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),  # Add agent_scratchpad placeholder
    ])
    return create_tool_calling_agent(llm, tools, prompt)


def create_insights_agent(llm):
    tools = [tavily_rag_search,process_search_tool]
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an insights generator. Provide detailed insights."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),  # Add agent_scratchpad placeholder
    ])
    return create_tool_calling_agent(llm, tools, prompt)

def create_general_purpose_agent(llm):
    tools = [tavily_rag_search]  # General-purpose tool like Tavily
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a general-purpose assistant. Answer any query comprehensively."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])
    return create_tool_calling_agent(llm, tools, prompt)


# Coordinator agent
def create_coordinator_agent(llm):
    tools = []  # No tools needed for the coordinator
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are the coordinator. Manage interactions between agents and combine their responses."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),  # Add agent_scratchpad placeholder
    ])
    return create_tool_calling_agent(llm, tools, prompt)

# Initialize agents (replace `llm` with your actual LLM instance)
stock_data_agent = create_stock_data_agent(llm)
sentiment_agent = create_sentiment_agent(llm)
insights_agent = create_insights_agent(llm)
coordinator_agent = create_coordinator_agent(llm)

# Agent executors
stock_data_executor = AgentExecutor(agent=stock_data_agent, tools=[get_stock_data, get_stock_analysis], verbose=True)
sentiment_executor = AgentExecutor(agent=sentiment_agent, tools=[get_market_sentiment_news], verbose=True)
insights_executor = AgentExecutor(agent=insights_agent, tools=[tavily_rag_search,process_search_tool], verbose=True)
coordinator_executor = AgentExecutor(agent=coordinator_agent, tools=[], verbose=True)


def classify_query(query):
    """Classifies the query type: stock/finance, sentiment, or general-purpose.
    Handles cases where the query contains keywords from multiple categories."""
    stock_keywords = ["stock", "market", "share", "nasdaq", "dow jones", "finance", "investment"]
    sentiment_keywords = ["sentiment", "news", "social media", "opinion", "trends"]
    
    is_stock = any(keyword in query.lower() for keyword in stock_keywords)
    is_sentiment = any(keyword in query.lower() for keyword in sentiment_keywords)
    
    if is_stock and is_sentiment:
        return "both"  # Handle cases where the query is relevant to both categories
    elif is_stock:
        return "stock"
    elif is_sentiment:
        return "sentiment"
    else:
        return "general"  # Default to Tavily for general queries
    
    
def generate_insights_prompt(query, query_type):
    """Generates a context-aware prompt for the insights agent based on the query type."""
    if query_type in ["stock", "both"]:
        return f"Generate a financial analysis and investment insights for {query}. Consider earnings reports, revenue trends, P/E ratio, and market positioning."
    elif query_type == "sentiment":
        return f"Provide insights based on the sentiment analysis for {query}. Summarize key trends, opinions, and potential implications."
    elif query_type == "general":
        return f"Provide detailed insights and analysis for {query}. Consider relevant facts, trends, and context."
    else:
        return f"Provide insights and analysis for {query}."

        
def multi_agent_query(query):
    responses = []
    errors = []
    query_type = classify_query(query)
    print(query_type)


    # Fetch stock data if it's a stock-related query
    if query_type in ["stock", "both"]:
        try:
            stock_data_response = stock_data_executor.invoke(
                {"input": f"Retrieve the latest stock data and market trends for {query}. Provide key statistics, including open, high, low, close, and volume."}
            )
            responses.append(f"**Stock Data Analysis**:\n{stock_data_response['output']}")
        except Exception as e:
            errors.append(f"❌ Stock Data Agent failed: {str(e)}")

    # Fetch sentiment analysis if it's related to financial sentiment
    if query_type in ["sentiment", "both"]:
        try:
            sentiment_response = sentiment_executor.invoke(
                {"input": f"Analyze the market sentiment for {query}. Summarize the tone of recent news articles, social media discussions, and investor opinions."}
            )
            responses.append(f"**Sentiment Analysis**:\n{sentiment_response['output']}")
        except Exception as e:
            errors.append(f"❌ Sentiment Agent failed: {str(e)}")
            
    # Step 3: Generate insights (generic input)
    if query_type in ["stock", "sentiment", "both", "general"]:
        try:
            insights_prompt = generate_insights_prompt(query, query_type)
            insights_response = insights_executor.invoke({"input": insights_prompt})
            responses.append(f"**Insights**:\n{insights_response['output']}")
        except Exception as e:
            errors.append(f"❌ Insights Agent failed: {str(e)}")

    # Combine responses
    final_response = "\n\n".join(responses) if responses else "No data available."

    # Append errors if any agents failed
    if errors:
        final_response += "\n\n**Errors**:\n" + "\n".join(errors)

    return final_response


# Example
query = "Who are the current ICC T20 world cup champions?"
response = multi_agent_query(query)
print(response)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


general

Invoking: `tavily_search_tool` with `{'query': 'ICC T20 world cup champions'}`


[Document(metadata={'source': 'https://currentaffairs.adda247.com/t20-cricket-world-cup-winners-list/', 'title': ''}, page_content="ICC T20 World Cup Winners List. The T20 World Cup 2022 has concluded, and England emerged as the new champions of the ICC Men's T20 World Cup."), Document(metadata={'source': 'https://www.icc-cricket.com/', 'title': ''}, page_content="Get all the latest updates and exclusive content on ICC Women's Under-19 T20 World Cup 2025 ICC Women's Under-19 T20 World Cup, 2025 Dominant India secure back to back titles | Final | Match Highlights | U19WC2025 7d ICC Champions Trophy, 2025Every ICC Men's Champions Trophy 2025 squad 1h Tom Banton 11/11/1998England batter makes a return after almost five years for final India ODI 2h ICC Champions Trophy, 2025South Africa name replacement for Nortje ahead of Champions Trophy 2025 2h ICC World Test ChampionshipFinal WTC25 standings confi